In [1]:
import pandas

In [2]:
df = pandas.read_csv('transcripts.csv')

In [3]:
df.head()

,transcript,url
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...
3,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...
4,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...


In [4]:
df.dtypes

transcript    object
url           object
dtype: object

In [5]:
ted = pandas.read_csv('ted_main.csv')

## Combine and create new dataset 

In [6]:
df_new = pandas.merge(df, ted, on='url')

In [7]:
df_new.shape

(2467, 18)

In [8]:
#check n/a values
df_new.isna().sum()

transcript            0
url                   0
comments              0
description           0
duration              0
event                 0
film_date             0
languages             0
main_speaker          0
name                  0
num_speaker           0
published_date        0
ratings               0
related_talks         0
speaker_occupation    6
tags                  0
title                 0
views                 0
dtype: int64

## How many words in each transcript (length of talk)

In [9]:
df_new['num_words'] = df_new.transcript.apply(lambda x: len(x))

In [10]:
df_new.num_words.describe()

count     2467.000000
mean     11438.532631
std       5291.779202
min         17.000000
25%       7568.000000
50%      11539.000000
75%      15119.000000
max      51355.000000
Name: num_words, dtype: float64

In [11]:
df_new.duration.describe()

count    2467.000000
mean      821.760032
std       334.090010
min       135.000000
25%       582.000000
50%       852.000000
75%      1047.000000
max      3608.000000
Name: duration, dtype: float64

In [12]:
import ast

In [15]:
df_new.ratings = df_new.ratings.apply(lambda x: ast.literal_eval(x))

In [16]:
type(df_new.ratings[0])

list

In [20]:
# convert dataframe into dictionary for further usage
ted_talks = df_new.to_dict('records')

In [21]:
import json

In [22]:
action_list = []
for i in range(len(df_new)):
    # create index for each entry
    action = { "index" : { "_index" : "ted", "_type" : "_doc", "_id" : i+1 } }
    action_list.append(action)
    

In [23]:
with open('ted-talk.json', 'w') as outfile:
    for (idx, line) in zip(action_list, ted_talks):
        json.dump(idx, outfile)
        outfile.write('\n')
        json.dump(line, outfile)
        outfile.write('\n')